In [22]:
from scapy.all import * # Packet manipulation
import pandas as pd # Pandas - Create and Manipulate DataFrames
import numpy as np # Math Stuff (don't worry only used for one line :] )
import binascii # Binary to Ascii 
import seaborn as sns
sns.set(color_codes=True)
%matplotlib inline

Captura de 100 paquetes

In [23]:
num_of_packets_to_sniff = 100
pcap = sniff(count=num_of_packets_to_sniff)


<class 'scapy.plist.PacketList'>
100
<Sniffed: TCP:94 UDP:6 ICMP:0 Other:0>


<Ether  dst=ff:ff:ff:ff:ff:ff src=00:09:0f:09:00:22 type=IPv4 |<IP  version=4 ihl=5 tos=0x10 len=406 id=0 flags= frag=0 ttl=16 proto=udp chksum=0x689e src=192.168.128.1 dst=255.255.255.255 |<UDP  sport=bootps dport=bootpc len=386 chksum=0xf9fd |<BOOTP  op=BOOTREPLY htype=1 hlen=6 hops=0 xid=2984760539 secs=0 flags=B ciaddr=0.0.0.0 yiaddr=192.168.136.214 siaddr=0.0.0.0 giaddr=0.0.0.0 chaddr=b'P/\x9b\x014k\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' sname=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' file=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

In [25]:
analisis_paquetes = rdpcap("datasets/analisis_paquetes.pcap")


<class 'scapy.plist.PacketList'>
62
<analisis_paquetes.pcap: TCP:0 UDP:62 ICMP:0 Other:0>


<Ether  dst=80:37:73:96:9b:db src=88:e9:fe:6a:92:52 type=IPv4 |<IP  version=4 ihl=5 tos=0x0 len=961 id=1 flags= frag=0 ttl=64 proto=udp chksum=0x52e6 src=10.1.10.53 dst=84.54.22.33 |<UDP  sport=domain dport=domain len=941 chksum=0xf60e |<DNS  id=12 qr=0 opcode=QUERY aa=0 tc=0 rd=1 ra=0 z=0 ad=0 cd=0 rcode=ok qdcount=1 ancount=0 nscount=0 arcount=0 qd=<DNSQR  qname='google.com.' qtype=AAAA qclass=IN |> an=None ns=None ar=None |<Raw  load='�PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01b\x00\x00\x00�\x08\x06\x00\x00\x00(�TR\x00\x00:�IDATx��\t|T�����;K\x12\x08;\x08�\nE��$\x19��Ziߪ-�;N2��V�V�a�Z\x11!��\x01ܻik[�.ok�e�\x01\\��]�-���d�\x08�\x08��g���<��L\x12�$3wf��\x02<_\r�Y�r��y���\x1c\r\x18�8c�ew\r\x1e����\x13�iaK����u}Ųw�d��\x15�"`\x06:\x13�\x0b�\x0c\x16�\x12�&�9Ɛ�\x00���\x08\x111ҌQHP*�@\x04O4���7<y�\x0e.9���a2@a��&"�\x13\x11�c�kO�s@�9�i�4X�x�K�a!f��0w��3dx\x1d"���C(�}�0|q�*��\x0b�a!f�4��.�sKg\x10\x11]=9NԂ\x06��u���K_�e\x06*���\x19Px��%��\x03�3��\x1e"E?\x0f���+H�]Q�k����8�"�\x1d\x114v�2\x10��

In [26]:
# add .pcap to sniffed packets
for pkt in analisis_paquetes:
    pcap.append(pkt)

<class 'scapy.plist.PacketList'>
162
<Sniffed: TCP:94 UDP:68 ICMP:0 Other:0>


<Ether  dst=ff:ff:ff:ff:ff:ff src=00:09:0f:09:00:22 type=IPv4 |<IP  version=4 ihl=5 tos=0x10 len=406 id=0 flags= frag=0 ttl=16 proto=udp chksum=0x689e src=192.168.128.1 dst=255.255.255.255 |<UDP  sport=bootps dport=bootpc len=386 chksum=0xf9fd |<BOOTP  op=BOOTREPLY htype=1 hlen=6 hops=0 xid=2984760539 secs=0 flags=B ciaddr=0.0.0.0 yiaddr=192.168.136.214 siaddr=0.0.0.0 giaddr=0.0.0.0 chaddr=b'P/\x9b\x014k\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' sname=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' file=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

TCP

In [7]:
"""
0                   1                   2                   3
0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1 2 3 4 5 6 7 8 9 0 1
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|          Source Port          |       Destination Port        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                        Sequence Number                        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                    Acknowledgment Number                      |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|  Data |           |U|A|P|R|S|F|                               |
| Offset| Reserved  |R|C|S|S|Y|I|            Window             |
|       |           |G|K|H|T|N|N|                               |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|           Checksum            |         Urgent Pointer        |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                    Options                    |    Padding    |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
|                             data                              |
+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+-+
02 04 05 a0 01 03 03 05 01 01 08 0a 1d 74 65 c5 00 00 00 00 04 02 00 00
"""

print()

Exploración

In [6]:
#Recordemos que en los modelos de red, las capas inferiores encapsulan la información de las capas superiores
#Frames en paquetes, paquetes en segmentos (Ethernet, IP), modelo TCP/IP

ethernet_frame = pcap[0]
ip_packet = ethernet_frame.payload
segment = ip_packet.payload
data = segment.payload #recupera la informacion de la capa 4

print("Informacion del frame:")
print(ethernet_frame.summary())

print("Informacion del paquete:")
print(ip_packet.summary())

print("Informacion del segmento:")
print(segment.summary())

print("Data:")
print(data.summary())

ethernet_frame.show()
data.show()

Informacion del frame:
Ether / IP / TCP 172.28.0.2:8080 > 172.28.0.1:52420 FA
Informacion del paquete:
IP / TCP 172.28.0.2:8080 > 172.28.0.1:52420 FA
Informacion del segmento:
TCP 172.28.0.2:8080 > 172.28.0.1:52420 FA
Data:

###[ Ethernet ]### 
  dst       = 02:42:04:54:f1:51
  src       = 02:42:ac:1c:00:02
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 52
     id        = 9466
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = 6
     chksum    = 0xbd8e
     src       = 172.28.0.2
     dst       = 172.28.0.1
     \options   \
###[ TCP ]### 
        sport     = 8080
        dport     = 52420
        seq       = 293036563
        ack       = 2738177287
        dataofs   = 8
        reserved  = 0
        flags     = FA
        window    = 501
        chksum    = 0x5862
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (2326413667, 1059821441))]



Tipos de datos en Scapy

In [7]:
print(type(ethernet_frame))
print(type(ip_packet))
print(type(segment))

# Packets can be filtered on layers ie) ethernet_frame[scapy.layers.l2.Ether]
ethernet_type = type(ethernet_frame)
ip_type = type(ip_packet)
tcp_type = type(segment)
print("Ethernet",pcap[ethernet_type])
print("IP", pcap[ip_type])
print("TCP", pcap[tcp_type])

# Scapy provides this via import statements
from scapy.layers.l2 import Ether
from scapy.layers.inet import IP
from scapy.layers.inet import TCP, UDP

print("UDP", pcap[UDP])

<class 'scapy.layers.l2.Ether'>
<class 'scapy.layers.inet.IP'>
<class 'scapy.layers.inet.TCP'>
Ethernet <Ether from Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>
IP <IP from Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>
TCP <TCP from Sniffed: TCP:10 UDP:0 ICMP:0 Other:0>
UDP <UDP from Sniffed: TCP:0 UDP:0 ICMP:0 Other:0>


Conversion de PCAP a DF

In [34]:
# Obtenemos los campos de IP/TCP/UDP que seran las columnas de nuestro DF
ip_fields = [field.name for field in IP().fields_desc]
tcp_fields = [field.name for field in TCP().fields_desc]
udp_fields = [field.name for field in UDP().fields_desc]

#Concatenamos todos los campos
dataframe_fields = ip_fields + ['time'] + tcp_fields + ['payload','payload_raw','payload_hex']

# Creamos un DF en blanco con los campos como columnas
df = pd.DataFrame(columns=dataframe_fields)
for packet in pcap[IP]:
    # Obtenemos los valores de los campos de cada fila
    field_values = []
    # A;adimos todos los valores de los campos de IP al df
    for field in ip_fields:
        if field == 'options':
            # Obtenemos el numero de opciones del IP Header
            field_values.append(len(packet[IP].fields[field]))
        else:
            field_values.append(packet[IP].fields[field])
    
    field_values.append(packet.time)
    
    layer_type = type(packet[IP].payload)
    for field in tcp_fields:
        try:
            if field == 'options':
                field_values.append(len(packet[layer_type].fields[field]))
            else:
                field_values.append(packet[layer_type].fields[field])
        except:
            field_values.append(None)
    
    # Agregamos el payload
    field_values.append(len(packet[layer_type].payload))
    field_values.append(packet[layer_type].payload.original)
    field_values.append(binascii.hexlify(packet[layer_type].payload.original))
    # Agregamos la fila al df
    df_append = pd.DataFrame([field_values], columns=dataframe_fields)
    df = pd.concat([df, df_append], axis=0)

# Reset Index
df = df.reset_index()
# Drop old index column
df = df.drop(columns="index")

print(df.columns)
df.head(5)

Index(['version', 'ihl', 'tos', 'len', 'id', 'flags', 'frag', 'ttl', 'proto',
       'chksum', 'src', 'dst', 'options', 'time', 'sport', 'dport', 'seq',
       'ack', 'dataofs', 'reserved', 'flags', 'window', 'chksum', 'urgptr',
       'options', 'payload', 'payload_raw', 'payload_hex'],
      dtype='object')


,version,ihl,tos,len,id,flags,frag,ttl,proto,chksum,...,dataofs,reserved,flags,window,chksum,urgptr,options,payload,payload_raw,payload_hex
0,4,5,16,406,0,,0,16,17,26782,...,None,None,None,None,63997,None,None,378,b'\x02\x01\x06\x00\xb1\xe7\xd4\xdb\x00\x00\x80...,b'02010600b1e7d4db0000800000000000c0a888d60000...
1,4,5,72,1133,24358,,0,123,6,34545,...,8,0,PA,1050,3061,0,3,1081,b'\x17\x03\x03\x00C\xd8\xb8s\x11\x1b\xd2\x8a\x...,b'1703030043d8b873111bd28ae829f390f7d6eff53362...
2,4,5,0,52,0,DF,0,64,6,58776,...,8,0,A,2075,57818,0,3,0,b'',b''
3,4,5,72,1133,24358,,0,123,6,34545,...,8,0,PA,1050,3061,0,3,1081,b'\x17\x03\x03\x00C\xd8\xb8s\x11\x1b\xd2\x8a\x...,b'1703030043d8b873111bd28ae829f390f7d6eff53362...
4,4,5,0,64,0,DF,0,64,6,58764,...,11,0,A,2092,22936,0,6,0,b'',b''


In [35]:
# Return Src Address, Dst Address, Src Port, Dst Port
df[['src','dst','sport','dport']]

,src,dst,sport,dport
0,192.168.128.1,255.255.255.255,67,68
1,35.186.224.25,192.168.144.175,443,49580
2,192.168.144.175,35.186.224.25,49580,443
3,35.186.224.25,192.168.144.175,443,49580
4,192.168.144.175,35.186.224.25,49580,443
...,...,...,...,...
157,84.54.22.33,10.1.10.53,53,53
158,10.1.10.53,84.54.22.33,53,53
159,84.54.22.33,10.1.10.53,53,53
160,10.1.10.53,84.54.22.33,53,53


In [36]:
#Calcular estadisticas
print("# Top Source Address")
print(df['src'].describe(),'\n\n')

# Top Source Address
count                 162
unique                 12
top       192.168.144.175
freq                   55
Name: src, dtype: object 




In [37]:
#¿Quien es la IP de origen mas frecuente?
frequent_address = df['src'].describe()['top']


# A que IPs les habla la IP mas frecuente
print(df[df['src'] == frequent_address]['dst'].unique(),"\n\n")

['35.186.224.25' '157.240.233.60' '192.168.191.255' '142.250.217.206'
 '192.168.8.200' '104.46.162.226' '172.217.165.202' '64.4.54.254'
 '170.114.15.35'] 


